In [1]:
import numpy as np
import cv2
import pandas as pd

# Load CSV file
csv_path ="ssd_images_1_to_17.csv"
dff = pd.read_csv(csv_path)

# Define parameters
num_frames = 12
height = 100
width = 96
channels = 3

# Create an empty NumPy array with the specified shape
sequence_of_frames = np.zeros((num_frames, height, width, channels))
main_arr = []
main_labels = []

# Iterate through each row in the DataFrame
cnt = 0
for index, row in dff.iterrows():
    # Load the image file
    img_path = row['filepath']
    frame_data = cv2.imread(img_path)
    #frame_data = cv2.cvtColor(frame_data, cv2.COLOR_BGR2RGB)
    
    # Store frame data in sequence_of_frames array
    sequence_of_frames[cnt] = frame_data
    cnt += 1
    
    # If sequence_of_frames array is filled, append it to main_arr and reset sequence_of_frames
    if cnt >= num_frames:
        main_arr.append(sequence_of_frames)
        main_labels.append(row['label'])
        cnt = 0
        sequence_of_frames = np.zeros((num_frames, height, width, channels))

# Print the number of sequences stored in main_arr
print("Number of sequences:", len(main_arr))
print("Number of label:", len(main_labels))

Number of sequences: 748
Number of label: 748


In [11]:
import numpy as np
import cv2
import pandas as pd

# Load CSV file
csv_path = "ssd_images_19_to_24.csv"
dff = pd.read_csv(csv_path)

# Define parameters
num_frames = 12
height = 100
width = 96
channels = 3

# Create an empty NumPy array with the specified shape
sequence_of_frames = np.zeros((num_frames, height, width, channels))
main_arr_test = []
main_labels_test = []

# Iterate through each row in the DataFrame
cnt = 0
for index, row in dff.iterrows():
    # Load the image file
    img_path = row['filepath']
    frame_data = cv2.imread(img_path)
    #frame_data = cv2.cvtColor(frame_data, cv2.COLOR_BGR2RGB)
    
    # Store frame data in sequence_of_frames array
    sequence_of_frames[cnt] = frame_data
    cnt += 1
    
    # If sequence_of_frames array is filled, append it to main_arr and reset sequence_of_frames
    if cnt >= num_frames:
        main_arr_test.append(sequence_of_frames)
        main_labels_test.append(row['label'])
        cnt = 0
        sequence_of_frames = np.zeros((num_frames, height, width, channels))

# Print the number of sequences stored in main_arr
print("Number of sequences:", len(main_arr_test))
print("Number of label:", len(main_labels_test))

Number of sequences: 264
Number of label: 264


In [12]:
main_arr_test=np.array(main_arr_test)
main_labels_test=np.array(main_labels_test)

In [9]:
import tensorflow as tf
print(tf.__version__)

2.3.0


  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0


In [10]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.layers import TimeDistributed
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed, Reshape
from tensorflow.keras.layers import TimeDistributed, BatchNormalization

# Define input shape and number of classes
input_frames = 12
height = 100
width = 96
channels = 3
num_classes = 6
input_shape = (input_frames, height, width, channels)

def create_crnn_model(input_shape, num_classes):
    model = Sequential()

    # Convolutional layers
    model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu'), input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Flatten()))

    # Fully connected layers
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))

    # LSTM layer
    model.add(LSTM(256, return_sequences=False))

    # Fully connected layer after LSTM
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

main_arr = np.array(main_arr)
main_labels = np.array(main_labels)


# Create the CRNN model
model = create_crnn_model(input_shape, num_classes)
# Convert your data to numpy arrays if needed
x_train_np = np.array(main_arr)
y_train_np = np.array(main_labels)



# Split the data using KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = []

for train_index, val_index in kf.split(main_arr):
    x_train, x_val = main_arr[train_index], main_arr[val_index]
    y_train, y_val = main_labels[train_index], main_labels[val_index]

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, epochs=20, batch_size=12, verbose=1)

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(x_val, y_val)
    cv_scores.append(test_accuracy)

# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))

# Evaluate the model on the test set
#test_loss, test_accuracy = model.evaluate(x_test, y_test)
#print("Test Loss:", test_loss)
#print("Test Accuracy:", test_accuracy)

# Make predictions
#predictions = model.predict(x_test)
#print("The predictions are:", predictions)

model.save('CRNN_model_video.h5')
print("the model saved successfully")


Epoch 1/20
50/50 [==============================] - 132s 3s/step - loss: 1.6042 - accuracy: 0.4666
Epoch 2/20
50/50 [==============================] - 123s 2s/step - loss: 0.9341 - accuracy: 0.6538
Epoch 3/20
50/50 [==============================] - 124s 2s/step - loss: 0.7879 - accuracy: 0.7174
Epoch 4/20
50/50 [==============================] - 123s 2s/step - loss: 0.5952 - accuracy: 0.7893
Epoch 5/20
50/50 [==============================] - 126s 3s/step - loss: 0.4390 - accuracy: 0.8478
Epoch 6/20
50/50 [==============================] - 123s 2s/step - loss: 0.4876 - accuracy: 0.8127
Epoch 7/20
50/50 [==============================] - 134s 3s/step - loss: 0.3923 - accuracy: 0.8612
Epoch 8/20
50/50 [==============================] - 176s 4s/step - loss: 0.3322 - accuracy: 0.8779
Epoch 9/20
50/50 [==============================] - 179s 4s/step - loss: 0.2607 - accuracy: 0.9114
Epoch 10/20
50/50 [==============================] - 166s 3s/step - loss: 0.2667 - accuracy: 0.9047
Epoch 11/

5/5 [==============================] - 6s 1s/step - loss: 0.6511 - accuracy: 0.8322
Epoch 1/20
50/50 [==============================] - 128s 3s/step - loss: 0.1165 - accuracy: 0.9649
Epoch 2/20
50/50 [==============================] - 126s 3s/step - loss: 0.0438 - accuracy: 0.9833
Epoch 3/20
50/50 [==============================] - 120s 2s/step - loss: 0.0914 - accuracy: 0.9649
Epoch 4/20
50/50 [==============================] - 122s 2s/step - loss: 0.0903 - accuracy: 0.9716
Epoch 5/20
50/50 [==============================] - 123s 2s/step - loss: 0.0520 - accuracy: 0.9783
Epoch 6/20
50/50 [==============================] - 122s 2s/step - loss: 0.0476 - accuracy: 0.9866
Epoch 7/20
50/50 [==============================] - 122s 2s/step - loss: 0.0364 - accuracy: 0.9917
Epoch 8/20
50/50 [==============================] - 122s 2s/step - loss: 0.0310 - accuracy: 0.9933
Epoch 9/20
50/50 [==============================] - 122s 2s/step - loss: 0.0228 - accuracy: 0.9933
Epoch 10/20
50/50 [======

In [20]:
test_loss, test_accuracy = model.evaluate(main_arr_test, main_labels_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions
predictions = model.predict(main_arr_test)
print("The predictions are:", predictions)

9/9 [==============================] - 7s 829ms/step - loss: 3.5890 - accuracy: 0.2311
Test Loss: 3.5890049934387207
Test Accuracy: 0.23106060922145844
The predictions are: [[9.8772895e-01 7.7072473e-04 3.1163120e-03 2.5970912e-05 4.4901815e-04
  7.9090158e-03]
 [9.1466635e-01 7.8621440e-02 2.4507132e-03 1.1810027e-04 1.4477402e-04
  3.9985953e-03]
 [6.3559644e-02 7.7421373e-06 9.3585277e-01 4.1119420e-05 7.4398311e-05
  4.6436462e-04]
 ...
 [3.6787608e-04 4.6090076e-06 3.7525257e-01 1.7636995e-06 1.2947589e-05
  6.2436020e-01]
 [2.6711437e-04 1.6905942e-05 2.9317532e-02 3.5679616e-06 1.1948776e-05
  9.7038293e-01]
 [5.9208422e-05 2.7506494e-05 3.8807586e-02 6.3081870e-07 1.5337487e-06
  9.6110356e-01]]


In [13]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('CRNN_model_video.h5')

# Now you can use the loaded model for prediction or further training
# Assuming you have new data stored in a variable called 'new_data'
#predictions = loaded_model.predict(new_data)

# Use the predictions as 

predictions=model.predict(main_arr_test)


In [16]:
predictions=np.array(predictions)
y_pred=[]
for i in predictions:
    y_pred.append(np.argmax(i))
print(y_pred)    

[0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 1, 1, 5, 5, 5, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 5, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 4, 5, 2, 5, 5, 5, 4, 5, 1, 1, 5, 1, 5, 5, 1, 5, 2, 5, 5, 1, 2, 2, 2, 2, 1, 2, 5, 5, 5, 1, 5, 0, 2, 2, 5, 5, 5, 5, 0, 5, 1, 1, 1, 1, 5, 1, 1, 5, 5, 5, 4, 5, 2, 5, 2, 5, 5, 1, 2, 5, 1, 1, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 5, 2, 2, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 3, 1, 0, 1, 0, 5, 5, 3, 3, 3, 3, 1, 3, 1, 3, 1, 3, 5, 3, 2, 0, 1, 0, 0, 0, 0, 0, 0, 2, 5, 3, 3, 0, 2, 0, 5, 1, 5, 5, 5, 2, 5, 1, 2, 5, 5, 5, 0, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 2, 0, 2, 0, 0, 5, 0, 5, 1, 1, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 5, 2, 5, 5, 5, 5, 5, 2, 2, 5, 5, 5]


In [19]:
from sklearn.metrics import accuracy_score
test_accuracy=accuracy_score(main_labels_test,y_pred)
#print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.36742424242424243


In [ ]:
# Define emotion classes
emotion_classes = ['angry', 'calm', 'fearful', 'happy', 'neutral', 'sad']

# Make predictions
#predictions = loaded_model.predict()

# Get the index of the maximum probability for each prediction
max_prob_indices = np.argmax(predictions, axis=1)

# Map the indices to emotion classes
predicted_emotions = [emotion_classes[idx] for idx in max_prob_indices]

# Print the predicted emotions
print("Predicted Emotions:")
for emotion in predicted_emotions:
    print(emotion)


### CRNN with k_fold=10

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.layers import TimeDistributed
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed, Reshape
from tensorflow.keras.layers import TimeDistributed, BatchNormalization

# Define input shape and number of classes
input_frames = 12
height = 96
width = 216
channels = 3
num_classes = 6
input_shape = (input_frames, height, width, channels)

def create_crnn_model(input_shape, num_classes):
    model = Sequential()

    # Convolutional layers
    model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu'), input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Flatten()))

    # Fully connected layers
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))

    # LSTM layer
    model.add(LSTM(256, return_sequences=False))

    # Fully connected layer after LSTM
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Create the CRNN model
model = create_crnn_model(input_shape, num_classes)
# Convert your data to numpy arrays if needed
x_train_np = np.array(main_arr)
y_train_np = np.array(main_labels)

main_arr = np.array(main_arr)
main_labels = np.array(main_labels)


# Split the data using KFold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

cv_scores = []

for train_index, val_index in kf.split(main_arr):
    x_train, x_val = main_arr[train_index], main_arr[val_index]
    y_train, y_val = main_labels[train_index], main_labels[val_index]

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, epochs=20, batch_size=12, verbose=1)

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(x_val, y_val)
    cv_scores.append(test_accuracy)

# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))

# Evaluate the model on the test set
#test_loss, test_accuracy = model.evaluate(x_test, y_test)
#print("Test Loss:", test_loss)
#print("Test Accuracy:", test_accuracy)

# Make predictions
#predictions = model.predict(x_test)
#print("The predictions are:", predictions)

model.save('CRNN_model_10.h5')
print("the model saved successfully")


Epoch 1/20
57/57 [==============================] - 101s 2s/step - loss: 1.6931 - accuracy: 0.3715
Epoch 2/20
57/57 [==============================] - 91s 2s/step - loss: 1.2347 - accuracy: 0.5171
Epoch 3/20
57/57 [==============================] - 91s 2s/step - loss: 0.9051 - accuracy: 0.6434
Epoch 4/20
57/57 [==============================] - 90s 2s/step - loss: 0.6388 - accuracy: 0.7637
Epoch 5/20
57/57 [==============================] - 90s 2s/step - loss: 0.5118 - accuracy: 0.8068
Epoch 6/20
57/57 [==============================] - 90s 2s/step - loss: 0.3390 - accuracy: 0.8782
Epoch 7/20
57/57 [==============================] - 89s 2s/step - loss: 0.3870 - accuracy: 0.8707
Epoch 8/20
57/57 [==============================] - 93s 2s/step - loss: 0.2945 - accuracy: 0.9064
Epoch 9/20
57/57 [==============================] - 90s 2s/step - loss: 0.2100 - accuracy: 0.9227
Epoch 10/20
57/57 [==============================] - 90s 2s/step - loss: 0.2922 - accuracy: 0.8915
Epoch 11/20
57/57 